In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
features_to_take = ['TransactionID', 'TransactionDT', 'TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3',
                    'card4', 'card5', 'card6', 'addr1', 'addr2', 'dist1', 'dist2', 'P_emaildomain', 'R_emaildomain',
                    'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D2',
                    'D3', 'D4', 'D5', 'D10', 'D11', 'D15', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'V1',
                    'V3', 'V5', 'V7', 'V11', 'V13', 'V14', 'V16', 'V20', 'V24', 'V26', 'V28', 'V30', 'V35', 'V36',
                    'V38', 'V40', 'V41', 'V45', 'V47', 'V49', 'V54', 'V55', 'V56', 'V58', 'V62', 'V65', 'V67', 'V68',
                    'V70', 'V76', 'V78', 'V80', 'V83', 'V87', 'V88', 'V89', 'V91', 'V107', 'V110', 'V112', 'V119',
                    'V121', 'V122', 'V125', 'V131', 'V138', 'V140', 'V142', 'V145', 'V147', 'V162', 'V169', 'V173', 
                    'V174', 'V185', 'V195', 'V210', 'V216', 'V220', 'V222', 'V241', 'V251', 'V271', 'V280', 'V281',
                    'V283', 'V284', 'V286', 'V287', 'V289', 'V291', 'V301', 'V303', 'V305', 'V311', 'V314', 'V323',
                    'V325', 'V335']

features_to_take_id = ['TransactionID', 'id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_09', 'id_10',
                       'id_11', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20',
                       'id_28', 'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37',
                       'id_38', 'DeviceType', 'DeviceInfo']

# ID attributes in 
features_to_take_id_test = [feature.replace('_', '-') for feature in features_to_take_id]

# Mention features encoded to a particular dtype
string_features_train = ['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'M4', 'id_12', 'id_15',
                         'id_16', 'id_28', 'id_29', 'id_30', 'id_31', 'id_33', 'id_34', 'DeviceType', 'DeviceInfo']

string_features_test = [feature.replace('_', '-') for feature in string_features_train]

boolean_features = ['M' + str(i) for i in range(1, 10) if i != 4] + ['id_' + str(i) for i in range(35, 39)]

In [3]:
# Read train and test dataset
train_df = pd.read_csv('../dataset/train_transaction.csv', usecols=features_to_take + ['isFraud'])
test_df = pd.read_csv('../dataset/test_transaction.csv', usecols=features_to_take)

train_df_id = pd.read_csv('../dataset/train_identity.csv', usecols=features_to_take_id)
test_df_id = pd.read_csv('../dataset/test_identity.csv', usecols=features_to_take_id_test)

# Left join dataset
train_df = train_df.merge(right=train_df_id, how='left', on='TransactionID')
test_df = test_df.merge(right=test_df_id, how='left', on='TransactionID')

In [4]:
test_df.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id-31,id-32,id-33,id-34,id-35,id-36,id-37,id-38,DeviceType,DeviceInfo
0,3663549,18403224,31.95,W,10409,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3663550,18403263,49.00,W,4272,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3663551,18403310,171.00,W,4476,574.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3663552,18403310,284.95,W,10989,360.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3663553,18403317,67.95,W,18018,452.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
renaming_mapper = dict(zip(features_to_take_id_test, features_to_take_id))

In [6]:
test_df.rename(mapper=renaming_mapper, axis='columns', inplace=True)

In [7]:
le = preprocessing.LabelEncoder()

concat_df = pd.concat([train_df, test_df])

for feature in string_features_train:
    train_df[feature] = train_df[feature].astype('str')
    test_df[feature] = test_df[feature].astype('str')
    concat_df[feature] = concat_df[feature].astype('str')
    le.fit(concat_df[feature])
    train_df[feature] = le.transform(train_df[feature])
    test_df[feature] = le.transform(test_df[feature])
    
for feature in boolean_features:
    train_df[feature] = train_df[feature].astype('bool')
    test_df[feature] = test_df[feature].astype('bool')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
train_df.head(n=10)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,4,13926,NaN,150.0,1,142.0,...,136,NaN,461,4,True,True,True,True,2,2740
1,2987001,0,86401,29.0,4,2755,404.0,150.0,2,102.0,...,136,NaN,461,4,True,True,True,True,2,2740
2,2987002,0,86469,59.0,4,4663,490.0,150.0,4,166.0,...,136,NaN,461,4,True,True,True,True,2,2740
3,2987003,0,86499,50.0,4,18132,567.0,150.0,2,117.0,...,136,NaN,461,4,True,True,True,True,2,2740
4,2987004,0,86506,50.0,1,4497,514.0,150.0,2,102.0,...,162,32.0,268,3,True,True,True,True,1,1565
5,2987005,0,86510,49.0,4,5937,555.0,150.0,4,226.0,...,136,NaN,461,4,True,True,True,True,2,2740
6,2987006,0,86522,159.0,4,12308,360.0,150.0,4,166.0,...,136,NaN,461,4,True,True,True,True,2,2740
7,2987007,0,86529,422.5,4,12695,490.0,150.0,4,226.0,...,136,NaN,461,4,True,True,True,True,2,2740
8,2987008,0,86535,15.0,1,2803,100.0,150.0,4,226.0,...,130,32.0,80,2,True,True,True,True,1,2693
9,2987009,0,86536,117.0,4,17399,111.0,150.0,2,224.0,...,136,NaN,461,4,True,True,True,True,2,2740


In [9]:
test_df.head(n=10)

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,3663549,18403224,31.95,4,10409,111.0,150.0,4,226.0,2,...,136,NaN,461,4,True,True,True,True,2,2740
1,3663550,18403263,49.00,4,4272,111.0,150.0,4,226.0,2,...,136,NaN,461,4,True,True,True,True,2,2740
2,3663551,18403310,171.00,4,4476,574.0,150.0,4,226.0,2,...,136,NaN,461,4,True,True,True,True,2,2740
3,3663552,18403310,284.95,4,10989,360.0,150.0,4,166.0,2,...,136,NaN,461,4,True,True,True,True,2,2740
4,3663553,18403317,67.95,4,18018,452.0,150.0,2,117.0,2,...,136,NaN,461,4,True,True,True,True,2,2740
5,3663554,18403323,57.95,4,12839,321.0,150.0,4,226.0,2,...,136,NaN,461,4,True,True,True,True,2,2740
6,3663555,18403350,87.00,4,16560,476.0,150.0,4,126.0,2,...,136,NaN,461,4,True,True,True,True,2,2740
7,3663556,18403387,390.00,4,15066,170.0,150.0,2,102.0,1,...,136,NaN,461,4,True,True,True,True,2,2740
8,3663557,18403405,103.95,4,2803,100.0,150.0,4,226.0,2,...,136,NaN,461,4,True,True,True,True,2,2740
9,3663558,18403416,117.00,4,12544,321.0,150.0,4,226.0,2,...,136,NaN,461,4,True,True,True,True,2,2740


In [10]:
train_df = train_df.fillna(-1)
test_df = test_df.fillna(-1)

In [11]:
idxT = train_df.index[:3*len(train_df)//4]
idxV = train_df.index[3*len(train_df)//4:]

In [12]:
clf = RandomForestClassifier(max_depth=10, random_state=0, class_weight='balanced', n_estimators=100, verbose=3, n_jobs=-1)

h = clf.fit(train_df.loc[:, train_df.columns != 'isFraud'].iloc[idxT], train_df['isFraud'].iloc[idxT])

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 100building tree 2 of 100building tree 3 of 100
building tree 4 of 100


building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100building tree 14 of 100

building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   20.9s


building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.4min finished


In [13]:
test_fraud = train_df.iloc[idxV][train_df['isFraud'] == 1]
test_not_fraud = train_df.iloc[idxV][train_df['isFraud'] == 0]

y_test_fraud = test_fraud.isFraud
y_test_not_fraud = test_not_fraud.isFraud
X_test_fraud = test_fraud.drop(labels='isFraud', axis='columns')
X_test_not_fraud = test_not_fraud.drop(labels='isFraud', axis='columns')

print("Accuracy on Fraud:", accuracy_score(y_test_fraud, clf.predict(X_test_fraud)))
print("Accuracy on Not Fraud:", accuracy_score(y_test_not_fraud, clf.predict(X_test_not_fraud)))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished


Accuracy on Fraud: 0.6572549019607843


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.3s


Accuracy on Not Fraud: 0.8928473708212018


[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.4s finished


In [14]:
y_test = clf.predict(test_df)
y_test

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    1.2s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    4.7s finished


array([0, 0, 0, ..., 0, 0, 0])

In [15]:
result_df = pd.DataFrame({
    "TransactionID": test_df.TransactionID, 
    "isFraud": y_test})

In [16]:
result_df.to_csv("random_forest.csv", index=False)